In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!mkdir ~/.kaggle
# copying api file
!cp drive/MyDrive/kaggle.json ~/.kaggle/
# modding permission
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
# downloading data using api command from the site
!kaggle datasets download -d ardikasatria/datasettanamanpadisumatera
# unzip the downloaded file
!unzip datasettanamanpadisumatera.zip

  0% 0.00/4.75k [00:00<?, ?B/s]
100% 4.75k/4.75k [00:00<00:00, 16.0MB/s]
Archive:  datasettanamanpadisumatera.zip
  inflating: Data_Tanaman_Padi_Sumatera_version_1.csv  


In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer

In [5]:
data = pd.read_csv('/content/Data_Tanaman_Padi_Sumatera_version_1.csv')
le = LabelEncoder()
data['Provinsi'] = le.fit_transform(data['Provinsi'])
data.head()

,Provinsi,Tahun,Produksi,Luas Panen,Curah hujan,Kelembapan,Suhu rata-rata
0,0,1993,1329536.0,323589.0,1627.0,82.00,26.06
1,0,1994,1299699.0,329041.0,1521.0,82.12,26.92
2,0,1995,1382905.0,339253.0,1476.0,82.72,26.27
3,0,1996,1419128.0,348223.0,1557.0,83.00,26.08
4,0,1997,1368074.0,337561.0,1339.0,82.46,26.31


In [6]:
x = data[['Provinsi','Tahun','Luas Panen','Curah hujan','Kelembapan','Suhu rata-rata']]
y = data[['Produksi']]
column = x.columns

In [ ]:
x

,Provinsi,Tahun,Luas Panen,Curah hujan,Kelembapan,Suhu rata-rata
0,0,1993,323589.00,1627.0,82.00,26.06
1,0,1994,329041.00,1521.0,82.12,26.92
2,0,1995,339253.00,1476.0,82.72,26.27
3,0,1996,348223.00,1557.0,83.00,26.08
4,0,1997,337561.00,1339.0,82.46,26.31
...,...,...,...,...,...,...
219,3,2016,390799.00,2317.6,79.40,26.45
220,3,2017,396559.00,1825.1,77.04,26.36
221,3,2018,511940.93,1385.8,76.05,25.50
222,3,2019,464103.42,1706.4,78.03,27.23


Model, KFold, and Standarization

In [7]:
scaler = MinMaxScaler()
cv = KFold(n_splits =5, shuffle=True, random_state=100)

In [8]:
mlr = MLPRegressor(random_state=1, max_iter=50000, learning_rate_init=0.05)

In [ ]:
scaler = MinMaxScaler()
cv = KFold(n_splits =5, shuffle=True, random_state=100)

mlr = MLPRegressor(random_state=1, max_iter=50000, learning_rate_init=0.05)

pca = PCA(n_components=1)

pipeline = Pipeline([('scaler', scaler), ('ct', ColumnTransformer([("pca", pca, [3, 4, 5]), ("pass", "passthrough", [0, 1, 2])])), ('model', mlr)])

R2_scores = cross_val_score(pipeline, x, y.values.ravel(), cv= cv)
MAE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv,scoring="neg_mean_absolute_error")
MSE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv, scoring="neg_mean_squared_error")
RMSE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv, scoring="neg_root_mean_squared_error")

for i, (train_index, test_index) in enumerate(cv.split(x)):
  print(f"Iteration {i+1}:")
  print(f"Score: R2 {R2_scores[i]}, MAE = {MAE_scores[i]}, MSE = {MSE_scores[i]}, RMSE = {RMSE_scores[i]}")

print("")
print("Final Scores:")
print(f"R2 = {R2_scores.mean()}")
print(f"MAE = {MAE_scores.mean()}")
print(f"MSE = {MSE_scores.mean()}")
print(f"RMSE = {RMSE_scores.mean()}")

In [ ]:
scaler = MinMaxScaler()
cv = KFold(n_splits =5, shuffle=True, random_state=100)

mlr = MLPRegressor(random_state=1, solver="lbfgs", max_iter=5000, learning_rate_init=0.05)

pca = PCA(n_components=1)

pipeline = Pipeline([('scaler', scaler), ('ct', ColumnTransformer([("pca", pca, [3, 4, 5]), ("pass", "passthrough", [0, 1, 2])])), ('model', mlr)])

R2_scores = cross_val_score(pipeline, x, y.values.ravel(), cv= cv)
MAE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv,scoring="neg_mean_absolute_error")
MSE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv, scoring="neg_mean_squared_error")
RMSE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv, scoring="neg_root_mean_squared_error")

for i, (train_index, test_index) in enumerate(cv.split(x)):
  print(f"Iteration {i+1}:")
  print(f"Score: R2 {R2_scores[i]}, MAE = {MAE_scores[i]}, MSE = {MSE_scores[i]}, RMSE = {RMSE_scores[i]}")

print("")
print("Final Scores:")
print(f"R2 = {R2_scores.mean()*100}")
print(f"MAE = {MAE_scores.mean()}")
print(f"MSE = {MSE_scores.mean()}")
print(f"RMSE = {RMSE_scores.mean()}")

Iteration 1:
Score: R2 0.8941103450119994, MAE = -223462.7162411657, MSE = -153757525679.44153, RMSE = -392119.2748124498
Iteration 2:
Score: R2 0.7207826228760033, MAE = -201462.54699794966, MSE = -282524533742.55804, RMSE = -531530.3695392748
Iteration 3:
Score: R2 0.9187263789816014, MAE = -228138.56539484905, MSE = -121537721227.18774, RMSE = -348622.6057317393
Iteration 4:
Score: R2 0.8723350481480745, MAE = -244190.609138666, MSE = -195361206972.17776, RMSE = -441996.84045497177
Iteration 5:
Score: R2 0.8952017608161457, MAE = -191770.8680948698, MSE = -99818705398.17693, RMSE = -315940.98404318636

Final Scores:
R2 = 86.02312311667649
MAE = -217805.0611735001
MSE = -170599938603.9084
RMSE = -406042.0149163244


In [ ]:
scaler = MinMaxScaler()
cv = KFold(n_splits =5, shuffle=True, random_state=100)

mlr = MLPRegressor(random_state=42, solver="lbfgs", max_iter=15000, learning_rate_init=0.05)

pca = PCA(n_components=1)

pipeline = Pipeline([('scaler', scaler), ('ct', ColumnTransformer([("pca", pca, [3, 4, 5]), ("pass", "passthrough", [0, 1, 2])])), ('model', mlr)])

R2_scores = cross_val_score(pipeline, x, y.values.ravel(), cv= cv)
MAE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv,scoring="neg_mean_absolute_error")
MSE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv, scoring="neg_mean_squared_error")
RMSE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv, scoring="neg_root_mean_squared_error")

for i, (train_index, test_index) in enumerate(cv.split(x)):
  print(f"Iteration {i+1}:")
  print(f"Score: R2 {R2_scores[i]}, MAE = {MAE_scores[i]}, MSE = {MSE_scores[i]}, RMSE = {RMSE_scores[i]}")

print("")
print("Final Scores:")
print(f"R2 = {R2_scores.mean()*100}")
print(f"MAE = {MAE_scores.mean()}")
print(f"MSE = {MSE_scores.mean()}")
print(f"RMSE = {RMSE_scores.mean()}")

Iteration 1:
Score: R2 0.8948988238530052, MAE = -197120.01707013592, MSE = -152612611611.51505, RMSE = -390656.6415812165
Iteration 2:
Score: R2 0.7834873100312201, MAE = -196482.36394537782, MSE = -219077148467.05872, RMSE = -468056.7791059742
Iteration 3:
Score: R2 0.9351058256427239, MAE = -201429.1906348212, MSE = -97043665256.62325, RMSE = -311518.322505472
Iteration 4:
Score: R2 0.8941898656959323, MAE = -190090.05307302234, MSE = -161917544695.48343, RMSE = -402389.7919871768
Iteration 5:
Score: R2 0.9012065036010456, MAE = -182633.2356647417, MSE = -94099280570.95038, RMSE = -306756.0603654806

Final Scores:
R2 = 88.17776657647855
MAE = -193550.97207761978
MSE = -144950050120.32617
RMSE = -375875.519109064


In [ ]:
for i in range(100):
  scaler = MinMaxScaler()
  cv = KFold(n_splits =5, shuffle=True, random_state=100)
  mlr = MLPRegressor(random_state=i, solver="lbfgs", max_iter=15000, learning_rate_init=0.001)
  pca = PCA(n_components=1)
  pipeline = Pipeline([('scaler', scaler), ('ct', ColumnTransformer([("pca", pca, [3, 4, 5]), ("pass", "passthrough", [0, 1, 2])])), ('model', mlr)])
  R2_scores = cross_val_score(pipeline, x, y.values.ravel(), cv= cv)
  print(f"R2 = {R2_scores.mean()*100} | {i}")

R2 = 74.46270147143468 | 0
R2 = 86.02312311667649 | 1
R2 = 84.39024930805174 | 2
R2 = 85.48837937410903 | 3
R2 = 87.01329184395581 | 4
R2 = 81.51644787528734 | 5
R2 = 85.18203861527599 | 6
R2 = 83.61514803700324 | 7
R2 = 87.60766558993596 | 8
R2 = 88.05206929571986 | 9
R2 = 86.27305667415331 | 10
R2 = 81.37920697393743 | 11
R2 = 80.98594082687273 | 12
R2 = 77.2613897556077 | 13
R2 = 85.69800455046197 | 14
R2 = 87.03825999116688 | 15
R2 = 80.11263050651017 | 16
R2 = 87.214749593199 | 17
R2 = 83.88938980016827 | 18
R2 = 84.48977493403174 | 19
R2 = 79.79431243265222 | 20
R2 = 79.43594255550144 | 21
R2 = 84.4229590204548 | 22
R2 = 84.312246793382 | 23
R2 = 80.5728640843639 | 24
R2 = 79.01442620291785 | 25
R2 = 85.8009857557357 | 26
R2 = 82.78127322263046 | 27
R2 = 74.4794473197014 | 28
R2 = 85.45925737502628 | 29
R2 = 87.03769779943403 | 30
R2 = 85.36929806576292 | 31
R2 = 82.03715916738302 | 32
R2 = 87.85540009112589 | 33
R2 = 82.7409013175217 | 34
R2 = 79.62495980036957 | 35
R2 = 84.4870

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


R2 = 87.27050433871287 | 84
R2 = 85.06888681401144 | 85
R2 = 85.03905796929706 | 86
R2 = 88.8182385415324 | 87
R2 = 84.31843709245412 | 88
R2 = 82.64333338023263 | 89
R2 = 82.72381215794698 | 90
R2 = 82.73739543500565 | 91
R2 = 82.72643618715108 | 92
R2 = 79.36895645398123 | 93
R2 = 80.79812221679475 | 94
R2 = 82.8202736010318 | 95
R2 = 81.57634513006865 | 96
R2 = 83.24894956880442 | 97
R2 = 83.29782126901515 | 98
R2 = 82.71502379937291 | 99


In [ ]:
for i in range(100, 500, 1):
  num = 2 + i*3
  scaler = MinMaxScaler()
  cv = KFold(n_splits =5, shuffle=True, random_state=100)
  mlr = MLPRegressor(random_state=num, solver="lbfgs", max_iter=15000, learning_rate_init=0.001)
  pca = PCA(n_components=1)
  pipeline = Pipeline([('scaler', scaler), ('ct', ColumnTransformer([("pca", pca, [3, 4, 5]), ("pass", "passthrough", [0, 1, 2])])), ('model', mlr)])
  R2_scores = cross_val_score(pipeline, x, y.values.ravel(), cv= cv)
  print(f"R2 = {R2_scores.mean()*100} | {num}")

R2 = 82.90487136244644 | 302
R2 = 83.96469071283354 | 305
R2 = 85.45465423103653 | 308
R2 = 83.28070392533729 | 311
R2 = 84.91994777613748 | 314
R2 = 80.05231349567448 | 317
R2 = 76.96587982991518 | 320
R2 = 85.7786003877307 | 323
R2 = 82.09859640651274 | 326
R2 = 84.33353098285059 | 329
R2 = 82.02047509769642 | 332
R2 = 79.70828940201066 | 335
R2 = 84.94823940036088 | 338
R2 = 83.488799036756 | 341
R2 = 86.10401403512202 | 344
R2 = 88.54597005012059 | 347
R2 = 84.57645473745583 | 350
R2 = 83.93449616822211 | 353
R2 = 76.90674530550507 | 356
R2 = 84.40040231636793 | 359
R2 = 81.4526395255632 | 362
R2 = 82.84007107500804 | 365
R2 = 83.25131083240677 | 368
R2 = 81.22788359357665 | 371
R2 = 87.0187227835699 | 374
R2 = 83.78269128203912 | 377
R2 = 84.99263226569894 | 380
R2 = 86.61106155411306 | 383
R2 = 82.8684939219577 | 386
R2 = 89.1155974402649 | 389
R2 = 75.48345852261208 | 392
R2 = 86.64201104570462 | 395
R2 = 86.21400445557848 | 398
R2 = 85.3693752407942 | 401
R2 = 84.34837551993898

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


R2 = 87.84265248005477 | 1010
R2 = 87.33608862855768 | 1013
R2 = 85.23275462166828 | 1016
R2 = 85.55163908442228 | 1019
R2 = 81.88064146129624 | 1022
R2 = 83.30475918136962 | 1025
R2 = 80.54314975317214 | 1028
R2 = 84.45721470480211 | 1031
R2 = 85.34413832274417 | 1034
R2 = 87.69482607618778 | 1037
R2 = 74.9990631706364 | 1040
R2 = 84.4788243242156 | 1043
R2 = 83.65412333095324 | 1046
R2 = 84.24639087078405 | 1049
R2 = 79.39945279091175 | 1052
R2 = 80.70936897700962 | 1055
R2 = 85.44464581959448 | 1058
R2 = 87.06843139913659 | 1061
R2 = 76.8699694317009 | 1064
R2 = 87.37502528130734 | 1067
R2 = 85.4778910373583 | 1070
R2 = 79.63434058434225 | 1073
R2 = 83.58799462029914 | 1076
R2 = 82.97689179477942 | 1079
R2 = 83.17071123391247 | 1082
R2 = 89.677842783527 | 1085
R2 = 85.41485781410493 | 1088
R2 = 82.9892108495059 | 1091
R2 = 86.99393246555861 | 1094
R2 = 77.57190104403381 | 1097
R2 = 85.76180510835725 | 1100
R2 = 86.28826395330746 | 1103
R2 = 85.39461872248786 | 1106
R2 = 81.964517172

In [ ]:
scaler = MinMaxScaler()
cv = KFold(n_splits =5, shuffle=True, random_state=100)

mlr = MLPRegressor(random_state=44, solver="lbfgs", tol=1e-4, max_iter=15000, learning_rate_init=0.001)

pca = PCA(n_components=1)

pipeline = Pipeline([('scaler', scaler), ('ct', ColumnTransformer([("pca", pca, [3, 4, 5]), ("pass", "passthrough", [0, 1, 2])])), ('model', mlr)])

R2_scores = cross_val_score(pipeline, x, y.values.ravel(), cv= cv)
MAE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv,scoring="neg_mean_absolute_error")
MSE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv, scoring="neg_mean_squared_error")
RMSE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv, scoring="neg_root_mean_squared_error")

for i, (train_index, test_index) in enumerate(cv.split(x)):
  print(f"Iteration {i+1}:")
  print(f"Score: R2 {R2_scores[i]}, MAE = {MAE_scores[i]}, MSE = {MSE_scores[i]}, RMSE = {RMSE_scores[i]}")

print("")
print("Final Scores:")
print(f"R2 = {R2_scores.mean()*100}")
print(f"MAE = {MAE_scores.mean()}")
print(f"MSE = {MSE_scores.mean()}")
print(f"RMSE = {RMSE_scores.mean()}")

Iteration 1:
Score: R2 0.8701241390545493, MAE = -286919.66793219367, MSE = -188586798462.25427, RMSE = -434265.81544286245
Iteration 2:
Score: R2 0.8990228508038826, MAE = -168776.40883006764, MSE = -102173160886.81923, RMSE = -319645.36737894267
Iteration 3:
Score: R2 0.7709432957442823, MAE = -294574.3804283176, MSE = -342534632002.5244, RMSE = -585264.5829046248
Iteration 4:
Score: R2 0.8733844986255831, MAE = -213875.59154903347, MSE = -193755269641.92743, RMSE = -440176.4074117642
Iteration 5:
Score: R2 0.8071792280805827, MAE = -214390.0580837367, MSE = -183658809315.5545, RMSE = -428554.3248125662

Final Scores:
R2 = 84.4130802461776
MAE = -235707.2213646698
MSE = -202141734061.81598
RMSE = -441581.29959015205


In [ ]:
scaler = MinMaxScaler()
cv = KFold(n_splits =5, shuffle=True, random_state=100)

mlr = MLPRegressor(random_state=41, solver="lbfgs", tol=1e-4, max_iter=15000, learning_rate_init=0.001)

pca = PCA(n_components=1)

pipeline = Pipeline([('scaler', scaler), ('ct', ColumnTransformer([("pca", pca, [3, 4, 5]), ("pass", "passthrough", [0, 1, 2])])), ('model', mlr)])

R2_scores = cross_val_score(pipeline, x, y.values.ravel(), cv= cv)
MAE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv,scoring="neg_mean_absolute_error")
MSE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv, scoring="neg_mean_squared_error")
RMSE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv, scoring="neg_root_mean_squared_error")

for i, (train_index, test_index) in enumerate(cv.split(x)):
  print(f"Iteration {i+1}:")
  print(f"Score: R2 {R2_scores[i]}, MAE = {MAE_scores[i]}, MSE = {MSE_scores[i]}, RMSE = {RMSE_scores[i]}")

print("")
print("Final Scores:")
print(f"R2 = {R2_scores.mean()*100}")
print(f"MAE = {MAE_scores.mean()}")
print(f"MSE = {MSE_scores.mean()}")
print(f"RMSE = {RMSE_scores.mean()}")

Iteration 1:
Score: R2 0.888468801132266, MAE = -201363.65352349888, MSE = -161949353559.6826, RMSE = -402429.31498547987
Iteration 2:
Score: R2 0.7515182150121895, MAE = -250581.25012215815, MSE = -251424897584.45026, RMSE = -501422.87301682826
Iteration 3:
Score: R2 0.8820847889041016, MAE = -230473.1196893297, MSE = -176332072756.7194, RMSE = -419919.1264478428
Iteration 4:
Score: R2 0.9154453452772178, MAE = -204942.99907673188, MSE = -129391028329.51353, RMSE = -359709.6444766439
Iteration 5:
Score: R2 0.8680078891077844, MAE = -210361.04610894065, MSE = -125720448498.36952, RMSE = -354570.7947623006

Final Scores:
R2 = 86.11050078867117
MAE = -219544.41370413188
MSE = -168963560145.74704
RMSE = -407610.35073781904


In [ ]:
scaler = MinMaxScaler()
cv = KFold(n_splits =5, shuffle=True, random_state=100)

mlr = MLPRegressor(random_state=40, solver="lbfgs", tol=1e-4, max_iter=15000, learning_rate_init=0.001)

pca = PCA(n_components=1)

pipeline = Pipeline([('scaler', scaler), ('ct', ColumnTransformer([("pca", pca, [3, 4, 5]), ("pass", "passthrough", [0, 1, 2])])), ('model', mlr)])

R2_scores = cross_val_score(pipeline, x, y.values.ravel(), cv= cv)
MAE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv,scoring="neg_mean_absolute_error")
MSE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv, scoring="neg_mean_squared_error")
RMSE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv, scoring="neg_root_mean_squared_error")

for i, (train_index, test_index) in enumerate(cv.split(x)):
  print(f"Iteration {i+1}:")
  print(f"Score: R2 {R2_scores[i]}, MAE = {MAE_scores[i]}, MSE = {MSE_scores[i]}, RMSE = {RMSE_scores[i]}")

print("")
print("Final Scores:")
print(f"R2 = {R2_scores.mean()*100}")
print(f"MAE = {MAE_scores.mean()}")
print(f"MSE = {MSE_scores.mean()}")
print(f"RMSE = {RMSE_scores.mean()}")

Iteration 1:
Score: R2 0.8860305207881195, MAE = -201841.0934877928, MSE = -165489868944.97937, RMSE = -406804.46033073356
Iteration 2:
Score: R2 0.7123158084804129, MAE = -283144.320363965, MSE = -291091632302.24646, RMSE = -539529.0838335283
Iteration 3:
Score: R2 0.8255143710599487, MAE = -285233.6483923911, MSE = -260928274913.03424, RMSE = -510811.38878556166
Iteration 4:
Score: R2 0.9340099539995732, MAE = -200449.0060433371, MSE = -100982257446.39618, RMSE = -317777.05619883287
Iteration 5:
Score: R2 0.7065769597042691, MAE = -250489.26410134303, MSE = -279480917278.8978, RMSE = -528659.5476097049

Final Scores:
R2 = 81.28895228064647
MAE = -244231.4664777658
MSE = -219594590177.1108
RMSE = -460716.30735167227


In [ ]:
scaler = MinMaxScaler()
cv = KFold(n_splits =5, shuffle=True, random_state=100)

mlr = MLPRegressor(random_state=39, solver="lbfgs", tol=1e-4, max_iter=15000, learning_rate_init=0.001)

pca = PCA(n_components=1)

pipeline = Pipeline([('scaler', scaler), ('ct', ColumnTransformer([("pca", pca, [3, 4, 5]), ("pass", "passthrough", [0, 1, 2])])), ('model', mlr)])

R2_scores = cross_val_score(pipeline, x, y.values.ravel(), cv= cv)
MAE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv,scoring="neg_mean_absolute_error")
MSE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv, scoring="neg_mean_squared_error")
RMSE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv, scoring="neg_root_mean_squared_error")

for i, (train_index, test_index) in enumerate(cv.split(x)):
  print(f"Iteration {i+1}:")
  print(f"Score: R2 {R2_scores[i]}, MAE = {MAE_scores[i]}, MSE = {MSE_scores[i]}, RMSE = {RMSE_scores[i]}")

print("")
print("Final Scores:")
print(f"R2 = {R2_scores.mean()*100}")
print(f"MAE = {MAE_scores.mean()}")
print(f"MSE = {MSE_scores.mean()}")
print(f"RMSE = {RMSE_scores.mean()}")

Iteration 1:
Score: R2 0.8619500283203718, MAE = -246255.93675995208, MSE = -200456050857.67776, RMSE = -447723.18552614376
Iteration 2:
Score: R2 0.653922700839834, MAE = -268082.83971502027, MSE = -350176370078.46027, RMSE = -591757.0194585446
Iteration 3:
Score: R2 0.8674549203193104, MAE = -201403.9877391546, MSE = -198209784951.26062, RMSE = -445207.5751278954
Iteration 4:
Score: R2 0.9517623176058561, MAE = -166554.50842462966, MSE = -73816436832.17656, RMSE = -271691.80486753106
Iteration 5:
Score: R2 0.8787157888929928, MAE = -203672.5583277167, MSE = -115521339215.45705, RMSE = -339884.302690573

Final Scores:
R2 = 84.2761151195673
MAE = -217193.96619329468
MSE = -187635996387.00644
RMSE = -419252.7775341376


In [ ]:
scaler = MinMaxScaler()
cv = KFold(n_splits =5, shuffle=True, random_state=100)
mlr = MLPRegressor(solver="lbfgs",
                   activation="relu",
                   hidden_layer_sizes=(50,),
                   tol=1e-4,
                   max_iter=50000,
                   learning_rate="adaptive",
                   learning_rate_init=0.001,
                   alpha=0.0001,
                   max_fun=50000,
)

pipeline = Pipeline([('scaler', scaler), ('model', mlr)])

R2_scores = cross_val_score(pipeline, x, y.values.ravel(), cv=cv)
print(f"R2 = {R2_scores.mean()*100}")

R2 = 82.63787697290127


In [ ]:
scaler = MinMaxScaler()
cv = KFold(n_splits =5, shuffle=True, random_state=100)
mlr = MLPRegressor(solver="adam",
                   activation="relu",
                   hidden_layer_sizes=(100,),
                   tol=1e-3,
                   max_iter=50000,
                   learning_rate="adaptive",
                   learning_rate_init=0.05,
                   alpha=0.0001,
)

pipeline = Pipeline([('scaler', scaler), ('model', mlr)])

R2_scores = cross_val_score(pipeline, x, y.values.ravel(), cv=cv)
print(f"R2 = {R2_scores.mean()*100}")

R2 = 85.46251168288725


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
for i in range(10):
  R2_scores = cross_val_score(pipeline, x, y.values.ravel(), cv= cv)
  print(f"R2 = {R2_scores.mean()}")

R2 = 0.8228824738356263
R2 = 0.7770630982156568
R2 = 0.7977959084423568
R2 = 0.8484579375157336
R2 = 0.8139390991480525
R2 = 0.8124282395040574
R2 = 0.8396410327935036
R2 = 0.7433634519672216
R2 = 0.8039880231235182
R2 = 0.7237659198003893


Base Accuracy

In [9]:
pipeline = Pipeline([('scaler', scaler), ('model', mlr)])

R2_scores = cross_val_score(pipeline, x, y.values.ravel(), cv= cv)
MAE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv,scoring="neg_mean_absolute_error")
MSE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv, scoring="neg_mean_squared_error")
RMSE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv, scoring="neg_root_mean_squared_error")

for i, (train_index, test_index) in enumerate(cv.split(x)):
  print(f"Iteration {i+1}:")
  print(f"Score: R2 {R2_scores[i]}, MAE = {MAE_scores[i]}, MSE = {MSE_scores[i]}, RMSE = {RMSE_scores[i]}")
print("")
print("Final Scores:")
print(f"R2 = {R2_scores.mean()}")
print(f"MAE = {MAE_scores.mean()}")
print(f"MSE = {MSE_scores.mean()}")
print(f"RMSE = {RMSE_scores.mean()}")

Iteration 1:
Score: R2 0.8532970782399653, MAE = -259025.3080893159, MSE = -213020603970.46115, RMSE = -461541.5517268853
Iteration 2:
Score: R2 0.7996535340043788, MAE = -233138.1472687494, MSE = -202719445599.71136, RMSE = -450243.7624217701
Iteration 3:
Score: R2 0.8856565953293425, MAE = -230493.53121384085, MSE = -170990742960.54684, RMSE = -413510.2694741049
Iteration 4:
Score: R2 0.8846085609999313, MAE = -239998.22631346236, MSE = -176579479883.0672, RMSE = -420213.6122058247
Iteration 5:
Score: R2 0.8552310113503626, MAE = -205994.76358421537, MSE = -137890227367.8325, RMSE = -371335.73403031455

Final Scores:
R2 = 0.855689355984796
MAE = -233729.99529391676
MSE = -180240099956.32382
RMSE = -423368.9859717799


In [ ]:
pipeline = Pipeline([('scaler', scaler), ('model', mlr)])

R2_scores = cross_val_score(pipeline, x, y.values.ravel(), cv= cv)
MAE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv,scoring="neg_mean_absolute_error")
MSE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv, scoring="neg_mean_squared_error")
RMSE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv, scoring="neg_root_mean_squared_error")

for i, (train_index, test_index) in enumerate(cv.split(x)):
  print(f"Iteration {i+1}:")
  print(f"Score: R2 {R2_scores[i]}, MAE = {MAE_scores[i]}, MSE = {MSE_scores[i]}, RMSE = {RMSE_scores[i]}")
print("")
print("Final Scores:")
print(f"R2 = {R2_scores.mean()}")
print(f"MAE = {MAE_scores.mean()}")
print(f"MSE = {MSE_scores.mean()}")
print(f"RMSE = {RMSE_scores.mean()}")

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50000) reached and the optimization hasn't converged yet.
  warnings.warn(


Iteration 1:
Score: R2 0.8532970269223679, MAE = -259025.33036123638, MSE = -213020679013.77136, RMSE = -461541.6411373159
Iteration 2:
Score: R2 0.7996535741581319, MAE = -233138.12843509723, MSE = -202719404936.0862, RMSE = -450243.7163187106
Iteration 3:
Score: R2 0.8856566018474912, MAE = -230493.56665532832, MSE = -170990733098.1808, RMSE = -413510.2577282183
Iteration 4:
Score: R2 0.8846086043492745, MAE = -239998.2003456205, MSE = -176579413232.28467, RMSE = -420213.5322339511
Iteration 5:
Score: R2 0.8368625024149484, MAE = -221760.51822463053, MSE = -158671665774.19388, RMSE = -395033.6754545572

Final Scores:
R2 = 0.8520156619384428
MAE = -236883.1488043826
MSE = -184396379210.90338
RMSE = -428108.56457455066


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
pipeline = Pipeline([('scaler', scaler), ('model', mlr)])

R2_scores = cross_val_score(pipeline, x, y.values.ravel(), cv= cv)
MAE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv,scoring="neg_mean_absolute_error")
MSE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv, scoring="neg_mean_squared_error")
RMSE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv, scoring="neg_root_mean_squared_error")

for i, (train_index, test_index) in enumerate(cv.split(x)):
  print(f"Iteration {i+1}:")
  print(f"Score: R2 {R2_scores[i]}, MAE = {MAE_scores[i]}, MSE = {MSE_scores[i]}, RMSE = {RMSE_scores[i]}")
print("")
print("Final Scores:")
print(f"R2 = {R2_scores.mean()}")
print(f"MAE = {MAE_scores.mean()}")
print(f"MSE = {MSE_scores.mean()}")
print(f"RMSE = {RMSE_scores.mean()}")

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50000) reached and the optimization hasn't converged yet.
  warnings.warn(


Iteration 1:
Score: R2 0.8532970272727438, MAE = -259025.34374833055, MSE = -213020659508.783, RMSE = -461541.64149813633
Iteration 2:
Score: R2 0.7996535747245774, MAE = -233138.12920086525, MSE = -202719405196.46085, RMSE = -450243.716858448
Iteration 3:
Score: R2 0.885656601808071, MAE = -230493.56679000374, MSE = -170990733214.57373, RMSE = -413510.2565507179
Iteration 4:
Score: R2 0.8846086055651143, MAE = -239998.1998062139, MSE = -176579413126.1684, RMSE = -420213.5316660764
Iteration 5:
Score: R2 0.8557082757782211, MAE = -223202.52275581702, MSE = -156638653877.3163, RMSE = -394480.8658341309

Final Scores:
R2 = 0.8557848170297454
MAE = -237171.55246024608
MSE = -183989772984.66046
RMSE = -427998.0024815019


In [ ]:
pipeline = Pipeline([('scaler', scaler), ('model', mlr)])

R2_scores = cross_val_score(pipeline, x, y.values.ravel(), cv= cv)
MAE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv,scoring="neg_mean_absolute_error")
MSE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv, scoring="neg_mean_squared_error")
RMSE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv, scoring="neg_root_mean_squared_error")

for i, (train_index, test_index) in enumerate(cv.split(x)):
  print(f"Iteration {i+1}:")
  print(f"Score: R2 {R2_scores[i]}, MAE = {MAE_scores[i]}, MSE = {MSE_scores[i]}, RMSE = {RMSE_scores[i]}")
print("")
print("Final Scores:")
print(f"R2 = {R2_scores.mean()}")
print(f"MAE = {MAE_scores.mean()}")
print(f"MSE = {MSE_scores.mean()}")
print(f"RMSE = {RMSE_scores.mean()}")

In [ ]:
pipeline = Pipeline([('scaler', scaler), ('model', mlr)])
for i in range(10):
  print(f"ITERATION {i}")
  R2_scores = cross_val_score(pipeline, x, y.values.ravel(), cv= cv)
  MAE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv,scoring="neg_mean_absolute_error")
  MSE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv, scoring="neg_mean_squared_error")
  RMSE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv, scoring="neg_root_mean_squared_error")

  for i, (train_index, test_index) in enumerate(cv.split(x)):
    print(f"Iteration {i+1}:")
    print(f"Score: R2 {R2_scores[i]}, MAE = {MAE_scores[i]}, MSE = {MSE_scores[i]}, RMSE = {RMSE_scores[i]}")
  print("")
  print("Final Scores:")
  print(f"R2 = {R2_scores.mean()}")
  print(f"MAE = {MAE_scores.mean()}")
  print(f"MSE = {MSE_scores.mean()}")
  print(f"RMSE = {RMSE_scores.mean()}")
  print("")

ITERATION 0


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50000) reached and the optimization hasn't converged yet.
  warnings.warn(


Iteration 1:
Score: R2 0.8532970339231779, MAE = -259025.3463918859, MSE = -213020687464.20312, RMSE = -461541.62007619883
Iteration 2:
Score: R2 0.7996535751622722, MAE = -233138.1287164358, MSE = -202719405955.3348, RMSE = -450243.7162297302
Iteration 3:
Score: R2 0.8856566019485519, MAE = -230493.56676319888, MSE = -170990730624.09598, RMSE = -413510.2577566531
Iteration 4:
Score: R2 0.8846086034401105, MAE = -239998.20140008337, MSE = -176579414326.77325, RMSE = -420213.5317701126
Iteration 5:
Score: R2 0.8360286775230585, MAE = -226209.6430092537, MSE = -153867497277.2424, RMSE = -399173.63217161415

Final Scores:
R2 = 0.8518488983994341
MAE = -237772.97725617155
MSE = -183435547129.52994
RMSE = -428936.5516008617

ITERATION 1


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50000) reached and the optimization hasn't converged yet.
  warnings.warn(


Iteration 1:
Score: R2 0.853297020117367, MAE = -259025.3457796996, MSE = -213020682307.3304, RMSE = -461541.63708561385
Iteration 2:
Score: R2 0.7996535714440702, MAE = -233138.1290406052, MSE = -202719405131.85806, RMSE = -450243.71741451044
Iteration 3:
Score: R2 0.8856566019774208, MAE = -230493.56613547768, MSE = -170990732967.56024, RMSE = -413510.25764318375
Iteration 4:
Score: R2 0.8846086032176221, MAE = -239998.2012959792, MSE = -176579414276.112, RMSE = -420213.53220712114
Iteration 5:
Score: R2 0.8343196690398578, MAE = -221789.99779054712, MSE = -156068512987.42026, RMSE = -387967.89938555536

Final Scores:
R2 = 0.8515070931592676
MAE = -236889.04800846177
MSE = -183875749534.0562
RMSE = -426695.40874719684

ITERATION 2


...

Iteration 1:
Score: R2 0.8532970354783335, MAE = -259025.34644467788, MSE = -213020687554.64597, RMSE = -461541.63122524286
Iteration 2:
Score: R2 0.7996535749441545, MAE = -233138.12808003012, MSE = -202719404796.18454, RMSE = -450243.71745024226
Iteration 3:
Score: R2 0.885656601817963, MAE = -230493.56669675442, MSE = -170990732975.7973, RMSE = -413510.2576596298
Iteration 4:
Score: R2 0.8846086033437818, MAE = -239998.20198267937, MSE = -176579415145.3902, RMSE = -420213.53258088056
Iteration 5:
Score: R2 0.8397693716366978, MAE = -232580.8700625105, MSE = -157515188922.96692, RMSE = -389133.6530693376

Final Scores:
R2 = 0.8525970374441861
MAE = -239047.22265333045
MSE = -184165085878.997
RMSE = -426928.5583970666

ITERATION 7


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50000) reached and the optimization hasn't converged yet.
  warnings.warn(


Iteration 1:
Score: R2 0.8532970216542859, MAE = -259025.34838138532, MSE = -213020675044.8081, RMSE = -461541.6388387815
Iteration 2:
Score: R2 0.7996535744604292, MAE = -233138.12804787038, MSE = -202719406539.9292, RMSE = -450243.71780638455
Iteration 3:
Score: R2 0.8856566017912728, MAE = -230493.56655345642, MSE = -170990733286.9362, RMSE = -413510.25774088915
Iteration 4:
Score: R2 0.884608603870794, MAE = -239998.20157299703, MSE = -176579412444.88867, RMSE = -420213.53191293246
Iteration 5:
Score: R2 0.8361497404997709, MAE = -218162.10545775073, MSE = -156397125451.03696, RMSE = -388803.05122325727

Final Scores:
R2 = 0.8518731084553105
MAE = -236163.47000269202
MSE = -183941470553.51984
RMSE = -426862.43950444896

ITERATION 8


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50000) reached and the optimization hasn't converged yet.
  warnings.warn(


Iteration 1:
Score: R2 0.8532970252036985, MAE = -259025.34923284053, MSE = -213020676367.77106, RMSE = -461541.6325064633
Iteration 2:
Score: R2 0.7996535743856905, MAE = -233138.1276976122, MSE = -202719404126.12796, RMSE = -450243.7165562456
Iteration 3:
Score: R2 0.8856566018595469, MAE = -230493.56594855365, MSE = -170990733155.37292, RMSE = -413510.257457475
Iteration 4:
Score: R2 0.884608607428197, MAE = -239998.20124675744, MSE = -176579414139.6939, RMSE = -420213.5352720863
Iteration 5:
Score: R2 0.8405833687316124, MAE = -221814.0245667199, MSE = -126260965104.698, RMSE = -387994.53832687356

Final Scores:
R2 = 0.852759835521749
MAE = -236893.85373849672
MSE = -177914238578.73276
RMSE = -426700.73602382874

ITERATION 9
Iteration 1:
Score: R2 0.8532970357617793, MAE = -259025.32802494318, MSE = -213020670384.75824, RMSE = -461541.6372114453
Iteration 2:
Score: R2 0.7996535741788386, MAE = -233138.12916648271, MSE = -202719404911.29892, RMSE = -450243.7159593881
Iteration 3:
Sc

Weather Variables

In [10]:
pca = PCA(n_components=1)

pipeline = Pipeline([('scaler', scaler), ('ct', ColumnTransformer([("pca", pca, [3, 4, 5]), ("pass", "passthrough", [0, 1, 2])])), ('model', mlr)])

R2_scores = cross_val_score(pipeline, x, y.values.ravel(), cv= cv)
MAE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv,scoring="neg_mean_absolute_error")
MSE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv, scoring="neg_mean_squared_error")
RMSE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv, scoring="neg_root_mean_squared_error")

for i, (train_index, test_index) in enumerate(cv.split(x)):
  print(f"Iteration {i+1}:")
  print(f"Score: R2 {R2_scores[i]}, MAE = {MAE_scores[i]}, MSE = {MSE_scores[i]}, RMSE = {RMSE_scores[i]}")

print("")
print("Final Scores:")
print(f"R2 = {R2_scores.mean()}")
print(f"MAE = {MAE_scores.mean()}")
print(f"MSE = {MSE_scores.mean()}")
print(f"RMSE = {RMSE_scores.mean()}")

Iteration 1:
Score: R2 0.8690497580485835, MAE = -249611.08920537328, MSE = -190146857989.62744, RMSE = -436058.3194821851
Iteration 2:
Score: R2 0.7950951613443877, MAE = -235373.8352247495, MSE = -207331809356.0576, RMSE = -455337.02831645217
Iteration 3:
Score: R2 0.887322450538935, MAE = -233071.1315750402, MSE = -168499599542.40768, RMSE = -410487.02725227224
Iteration 4:
Score: R2 0.8901433797703144, MAE = -235948.59502442347, MSE = -168109740462.271, RMSE = -410011.87844045565
Iteration 5:
Score: R2 0.8680924447257656, MAE = -237787.86379086404, MSE = -125639910577.2483, RMSE = -354457.20556542266

Final Scores:
R2 = 0.8619406388855972
MAE = -238358.5029640901
MSE = -171945583585.5224
RMSE = -413270.2918113576


In [ ]:
pca = PCA(n_components=1)
pipeline = Pipeline([('scaler', scaler), ('ct', ColumnTransformer([("pca", pca, [3, 4, 5]), ("pass", "passthrough", [0, 1, 2])])), ('model', mlr)])
for i in range(10):
  print(f"ITERATION {i}")
  R2_scores = cross_val_score(pipeline, x, y.values.ravel(), cv= cv)
  MAE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv,scoring="neg_mean_absolute_error")
  MSE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv, scoring="neg_mean_squared_error")
  RMSE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv, scoring="neg_root_mean_squared_error")

  for i, (train_index, test_index) in enumerate(cv.split(x)):
    print(f"Iteration {i+1}:")
    print(f"Score: R2 {R2_scores[i]}, MAE = {MAE_scores[i]}, MSE = {MSE_scores[i]}, RMSE = {RMSE_scores[i]}")
  print("")
  print("Final Scores:")
  print(f"R2 = {R2_scores.mean()}")
  print(f"MAE = {MAE_scores.mean()}")
  print(f"MSE = {MSE_scores.mean()}")
  print(f"RMSE = {RMSE_scores.mean()}")
  print("")

ITERATION 0
Iteration 1:
Score: R2 0.8690497792101867, MAE = -249611.07068348065, MSE = -190146827081.5604, RMSE = -436058.2835667893
Iteration 2:
Score: R2 0.7950951273014168, MAE = -235373.86950001834, MSE = -207331844065.512, RMSE = -455337.0662742753
Iteration 3:
Score: R2 0.8873221933739642, MAE = -233070.14157041273, MSE = -168497806373.63556, RMSE = -410479.5741278558
Iteration 4:
Score: R2 0.8901433598986543, MAE = -235948.66921530842, MSE = -168109771346.63287, RMSE = -410011.91611632914
Iteration 5:
Score: R2 0.8680924229360594, MAE = -237787.91273899467, MSE = -125639931800.11005, RMSE = -354457.23483929405

Final Scores:
R2 = 0.8619405765440563
MAE = -238358.33274164292
MSE = -171945236133.49017
RMSE = -413268.81498490873

ITERATION 1
Iteration 1:
Score: R2 0.8690497799341783, MAE = -249611.0703591821, MSE = -190146826431.9745, RMSE = -436058.28371228074
Iteration 2:
Score: R2 0.7950951271025002, MAE = -235373.87055499054, MSE = -207331844227.45612, RMSE = -455337.066082625

All Variables

In [11]:
pca = PCA(n_components=1)

pipeline = Pipeline([('scaler', scaler), ("pca", pca), ('model', mlr)])

R2_scores = cross_val_score(pipeline, x, y.values.ravel(), cv= cv)
MAE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv,scoring="neg_mean_absolute_error")
MSE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv, scoring="neg_mean_squared_error")
RMSE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv, scoring="neg_root_mean_squared_error")

for i, (train_index, test_index) in enumerate(cv.split(x)):
  print(f"Iteration {i+1}:")
  print(f"Score: R2 {R2_scores[i]}, MAE = {MAE_scores[i]}, MSE = {MSE_scores[i]}, RMSE = {RMSE_scores[i]}")
print("")
print("Final Scores:")
print(f"R2 = {R2_scores.mean()}")
print(f"MAE = {MAE_scores.mean()}")
print(f"MSE = {MSE_scores.mean()}")
print(f"RMSE = {RMSE_scores.mean()}")

Iteration 1:
Score: R2 0.672628227595291, MAE = -502944.24677792244, MSE = -475361580014.07587, RMSE = -689464.7054157855
Iteration 2:
Score: R2 0.4904060891152541, MAE = -499307.4746595164, MSE = -515629734630.81824, RMSE = -718073.6275834242
Iteration 3:
Score: R2 0.6568573351295387, MAE = -593737.1526472605, MSE = -513140389484.2998, RMSE = -716338.1809482863
Iteration 4:
Score: R2 0.6063864203192335, MAE = -583480.7679603385, MSE = -602333082741.9585, RMSE = -776101.2065072174
Iteration 5:
Score: R2 0.4599703124242206, MAE = -610008.9172825605, MSE = -514369942760.46075, RMSE = -717195.8886946165

Final Scores:
R2 = 0.5772496769167076
MAE = -557895.7118655196
MSE = -524166945926.32263
RMSE = -723434.721829866


In [ ]:
pca = PCA(n_components=1)
pipeline = Pipeline([('scaler', scaler), ("pca", pca), ('model', mlr)])
for i in range(10):
  print(f"ITERATION {i}")
  R2_scores = cross_val_score(pipeline, x, y.values.ravel(), cv= cv)
  MAE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv,scoring="neg_mean_absolute_error")
  MSE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv, scoring="neg_mean_squared_error")
  RMSE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv, scoring="neg_root_mean_squared_error")

  for i, (train_index, test_index) in enumerate(cv.split(x)):
    print(f"Iteration {i+1}:")
    print(f"Score: R2 {R2_scores[i]}, MAE = {MAE_scores[i]}, MSE = {MSE_scores[i]}, RMSE = {RMSE_scores[i]}")
  print("")
  print("Final Scores:")
  print(f"R2 = {R2_scores.mean()}")
  print(f"MAE = {MAE_scores.mean()}")
  print(f"MSE = {MSE_scores.mean()}")
  print(f"RMSE = {RMSE_scores.mean()}")
  print("")

ITERATION 0
Iteration 1:
Score: R2 0.6726282030348013, MAE = -502944.3108989189, MSE = -475361616086.6664, RMSE = -689464.7309689106
Iteration 2:
Score: R2 0.490406179234429, MAE = -499307.50243698375, MSE = -515629642362.708, RMSE = -718073.5625781161
Iteration 3:
Score: R2 0.6568573016238226, MAE = -593737.2258820168, MSE = -513140441496.5067, RMSE = -716338.2171949557
Iteration 4:
Score: R2 0.6063863853189434, MAE = -583480.7857999947, MSE = -602333134282.4813, RMSE = -776101.2405008567
Iteration 5:
Score: R2 0.45997029622668417, MAE = -610008.9780012682, MSE = -514369958570.81226, RMSE = -717195.8997851794

Final Scores:
R2 = 0.5772496730877361
MAE = -557895.7606038365
MSE = -524166958559.83496
RMSE = -723434.7302056036

ITERATION 1
Iteration 1:
Score: R2 0.6726282030972323, MAE = -502944.3095834727, MSE = -475361613741.597, RMSE = -689464.7315884149
Iteration 2:
Score: R2 0.49040618063982544, MAE = -499307.50148229906, MSE = -515629642103.42004, RMSE = -718073.5658523783
Iteration